In [13]:
import json

# import the smartpynector module
from NBs import smartpynector as sp

In [14]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://127.0.0.1:8000/subscriber"

In [15]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [16]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-03 22:12:30 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 22:12:31 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-03 22:12:31 INFO Request successful
2023-05-03 22:12:31 DEBUG 


<Response [200]>

In [17]:
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }
GRAPH_PATTERN= '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [18]:
ask_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
).json()["knowledgeInteractionId"]
ask_ki_id

2023-05-03 22:12:31 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 22:12:32 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-03 22:12:32 INFO Request successful
2023-05-03 22:12:32 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/148877f7-12ef-4042-8648-1cfd58931d08"}


'http://127.0.0.1:8000/subscriber/interaction/148877f7-12ef-4042-8648-1cfd58931d08'

In [19]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki_id,
    ask_data=[]
).json()["bindingSet"]

2023-05-03 22:12:32 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 22:12:32 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ask HTTP/1.1" 200 895
2023-05-03 22:12:32 INFO Request successful
2023-05-03 22:12:32 DEBUG {"bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measurements/58d8a74e-3f50-4428-bdcd-36aa1cb61809>","timestamp":"\"2023-05-03T22:12:15+02:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/78d48491-e9b9-42a8-b38a-93b702411811","exchangeStart":"2023-05-03T20:12:32.455+00:00","exchangeEnd":"2023-05-03T20:12:32.724+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://1

In [20]:
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=GRAPH_PATTERN,
    binding_set=binding_set,
    prefixes=PREFIXES,
    read_url=READ_URL,
    write_url=WRITE_URL,
)

2023-05-03 22:12:34 DEBUG Ignoring 500 server error
